# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a simple [linear regression model](https://scikit-learn.org/stable/modules/linear_model.html)
        - An advanced regressor of your choice (e.g., Random Forest, Neural Network, etc.)

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../data/adult/`, then you can use the code below to load them.

In [6]:
import pandas as pd

# reading the forestfires.csv file 
fires_dt = pd.read_csv('../../05_src/data/fires/forestfires.csv')

# re-assign names
fires_dt = fires_dt.rename(
    columns = {'X':'coord_x', 'Y':'coord_y', 'month': 'month', 'day':'day', 'FFMC': 'ffmc', 
               'DMC': 'dmc', 'DC':'dc', 'ISI': 'isi', 'temp': 'temp', 'RH': 'rh',
               'wind': 'wind', 'rain': 'rain', 'area': 'area'})

# exploring 
print(fires_dt.info())
print(fires_dt.describe())

# Get X and Y

Create the features data frame and target data.

In [ ]:
features = fires_dt.columns[fires_dt.columns != 'area']
area = fires_dt.columns[fires_dt.columns == 'area']

# data frame
X = fires_dt[features]

# data frame - target
Y= fires_dt[area]

print('Features data frame: ', X.shape)
print('Target data frame: ', Y.shape)
X.columns

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


In [ ]:
# importing libraries
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PowerTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV
import numpy as np
import pickle
import shap
import contextlib
import os

### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [7]:
# preproc1
# Define column names for different types of features
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# numerical
nums1 = Pipeline([ 
    ('scaler', StandardScaler(),) 
])

# categorical
cat1 = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore')) 
])

preproc1 = ColumnTransformer([
    ('num_trasnforms', nums1, numeric_features), 
    ('cat_trasnforms', cat1, categorical_features) 
])

preproc1

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [8]:
# preproc2

nums1 = Pipeline([
    ('scaler', StandardScaler())  # Step 1: Scale features using StandardScaler
])

nums2 = Pipeline([
    ('scaler', StandardScaler()),  # Step 1: Scale features using StandardScaler
    ('transform', PowerTransformer(method='yeo-johnson', standardize=False, copy=False))  # Step 2: Transform using Yeo-Johnson
])

# Pipeline for categorical features
cat2 = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preproc2 = ColumnTransformer([
    ('num_transforms_nums2', nums1, ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi']),  # Apply nums2 to rest of the numeric variables
    ('num_transforms_nums1', nums2, ['temp', 'rh', 'wind', 'rain']),  # Apply nums1 to a subset of numeric variables
    ('cat_transforms', cat2, categorical_features)  # Apply cat2 to categorical features
])

preproc2

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [ ]:
# Pipeline A = preproc1 + baseline
# Create the regression pipeline with LinearRegression
Pipeline_A = Pipeline([
    ('preprocessing', preproc1),  # Preprocessing 1
    ('regressor', LinearRegression())  # LinearRegression as the regressor
])

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Perform cross-validation on training data
cv_results = cross_validate(Pipeline_A, X_train, Y_train, cv=5,
                            scoring=('neg_mean_squared_error', 'r2'),
                            return_train_score=True)

# Extract and print cross-validation results
train_rmse_cv = np.sqrt(-cv_results['train_neg_mean_squared_error'])
test_rmse_cv = np.sqrt(-cv_results['test_neg_mean_squared_error'])
mean_train_rmse = np.mean(train_rmse_cv)
mean_test_rmse = np.mean(test_rmse_cv)

train_r2_cv = cv_results['train_r2']
test_r2_cv = cv_results['test_r2']
mean_train_r2 = np.mean(train_r2_cv)
mean_test_r2 = np.mean(test_r2_cv)


The train RMSE of 43.440 indicates that, on average, the model’s predictions are off by approximately 43.440 units from the actual values in the training set. This suggests that the model has errors in predicting the target variable for the training data.

And the test RMSE of 41.177 indicates that, on average, the model’s predictions are off by approximately 41.177 units from the actual values in the test set. This suggests that the model is not preforming well making accurate predictions.

The negative 
 value (-0.316) for the test dataset and high RMSE values indicate that the model is not performing well. It is likely underfitting and unable to capture the underlying patterns in the data.

In [ ]:
# Pipeline B = preproc2 + baseline
# Create the regression pipeline with LinearRegression
Pipeline_B = Pipeline([
    ('preprocessing', preproc2),  # Preprocessing 2
    ('regressor', LinearRegression())  # LinearRegression as the regressor
])

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Perform cross-validation on training data
cv_results = cross_validate(Pipeline_B, X_train, Y_train, cv=5,
                            scoring=('neg_mean_squared_error', 'r2'),
                            return_train_score=True)

# Extract and print cross-validation results
train_rmse_cv = np.sqrt(-cv_results['train_neg_mean_squared_error'])
test_rmse_cv = np.sqrt(-cv_results['test_neg_mean_squared_error'])
mean_train_rmse = np.mean(train_rmse_cv)
mean_test_rmse = np.mean(test_rmse_cv)

train_r2_cv = cv_results['train_r2']
test_r2_cv = cv_results['test_r2']
mean_train_r2 = np.mean(train_r2_cv)
mean_test_r2 = np.mean(test_r2_cv)

In terms of the RSME results, pipeine B model's predictions have a relatively high error rate, with both training and test sets.

Same than the previous pipeline Mean Test 
 is negative (-0.24), this means that the model does not explain much of the variance in the data and is performing poorly in both test and training sets. Very similar than pipeline A.

In [10]:
# Pipeline C = preproc1 + advanced model
# Create the regression pipeline
Pipeline_C = Pipeline([
    ('preprocessing', preproc1),  # Preprocessing steps
    ('regressor', RandomForestRegressor())  # RandomForestRegressor as the regressor
])
Pipeline_C

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Perform cross-validation on training data
cv_results = cross_validate(Pipeline_C, X_train, Y_train, cv=5,
                            scoring=('neg_mean_squared_error', 'r2'),
                            return_train_score=True)

# Extract and print cross-validation results
train_rmse_cv = np.sqrt(-cv_results['train_neg_mean_squared_error'])
test_rmse_cv = np.sqrt(-cv_results['test_neg_mean_squared_error'])
mean_train_rmse = np.mean(train_rmse_cv)
mean_test_rmse = np.mean(test_rmse_cv)

train_r2_cv = cv_results['train_r2']
test_r2_cv = cv_results['test_r2']
mean_train_r2 = np.mean(train_r2_cv)
mean_test_r2 = np.mean(test_r2_cv)

Pipeline C perfor better than the previous two pipelines with a mean train RMSE value of 19.006, but the Mean Test RMSE of 48.999. While the training RMSE is relatively low, indicating good performance in predicting the training data, the test RMSE is much higher, suggesting that the model does not generalize well to new data and exhibits high variability in prediction errors across different folds.

In term of the 
 Mean Train R-squared was 0.820 suggests a good fit of the model to the training data. But was not the case for the testing set with an 
 Mean Train R-squared of -1.873. Again ratifying a goot fit for the testing set but not foor the testing set.

In [9]:
# Pipeline D = preproc2 + advanced model
# Create the regression pipeline
Pipeline_D = Pipeline([
    ('preprocessing', preproc2),  # Preprocessing steps
    ('regressor', RandomForestRegressor())  # RandomForestRegressor as the regressor
])
Pipeline_D

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Perform cross-validation on training data
cv_results = cross_validate(Pipeline_D, X_train, Y_train, cv=5,
                            scoring=('neg_mean_squared_error', 'r2'),
                            return_train_score=True)

# Extract and print cross-validation results
train_rmse_cv = np.sqrt(-cv_results['train_neg_mean_squared_error'])
test_rmse_cv = np.sqrt(-cv_results['test_neg_mean_squared_error'])
mean_train_rmse = np.mean(train_rmse_cv)
mean_test_rmse = np.mean(test_rmse_cv)

train_r2_cv = cv_results['train_r2']
test_r2_cv = cv_results['test_r2']
mean_train_r2 = np.mean(train_r2_cv)
mean_test_r2 = np.mean(test_r2_cv)


# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

Tuning Pipeline A

In [ ]:
#tunning Pipeline_A using grid search

#Parameter grid
param_grid_A = {
    'regressor__fit_intercept': [True, False],  # Whether to calculate the intercept
    'preprocessing__num_trasnforms__scaler__with_mean': [True, False],  # Centering the data
    'preprocessing__num_trasnforms__scaler__with_std': [True, False],   # Scaling to unit variance
}

#G to test all values for n_neighbors
pipe_gs_A = GridSearchCV(Pipeline_A, param_grid_A, cv=5, verbose=10)

#fit model to data
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        pipe_gs_A.fit(X, Y)


In [ ]:
# Print best parameters and best score
print("Best parameters: ", pipe_gs_A.best_params_)
print("Best score: ", pipe_gs_A.best_score_)

Tuning Pipeline B

In [ ]:
#Parameter grid
param_grid_B = {
    'regressor__fit_intercept': [True, False],  # Whether to calculate the intercept
    'preprocessing__num_transforms_nums2__scaler__with_mean': [True, False],  # Centering the data for num_transforms_nums2
    'preprocessing__num_transforms_nums2__scaler__with_std': [True, False],   # Scaling to unit variance for num_transforms_nums2
    'preprocessing__num_transforms_nums1__scaler__with_mean': [True, False],  # Centering the data for num_transforms_nums1
    'preprocessing__num_transforms_nums1__scaler__with_std': [True, False],    # Scaling to unit variance for num_transforms_nums1
    'preprocessing__num_transforms_nums1__transform__method': ['yeo-johnson', 'box-cox'],  # Method for PowerTransformer in num_transforms_nums1
}

#Gridsearch to test all values for n_neighbors
pipe_gs_B = GridSearchCV(Pipeline_B, param_grid_B, cv=5, verbose=10)

#fit model to data
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        pipe_gs_B.fit(X, Y)



In [ ]:
# Print best parameters and best score
print("Best parameters: ", pipe_gs_B.best_params_)
print("Best score: ", pipe_gs_B.best_score_)

Tuning Pipeline C

In [ ]:
# The parameter grid
param_grid = {
    'regressor__n_estimators': [50, 100, 200],          # Number of trees in the forest
    'regressor__max_features': ['auto', 'sqrt', 'log2'], # Number of features to consider at every split
    'regressor__max_depth': [None, 10, 20, 30],          # Maximum number of levels in tree
    'regressor__min_samples_split': [2, 5, 10],          # Minimum number of samples required to split a node
    'regressor__min_samples_leaf': [1, 2, 4],            # Minimum number of samples required at each leaf node
    'regressor__bootstrap': [True, False]                # Method of selecting samples for training each tree
}
#use gridsearch to test all values for n_neighbors
pipe_gs_C = GridSearchCV(Pipeline_C, param_grid, cv=5, verbose=10)

# Fit the grid search to the data
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        pipe_gs_C.fit(X, Y)



In [ ]:
# Print best parameters and best score
print("Best parameters: ", pipe_gs_C.best_params_)
print("Best score: ", pipe_gs_C.best_score_)

# Evaluate

+ Which model has the best performance?

Pipeline C with RandomForestRegressor

Best parameters: {'regressor__bootstrap': True, 'regressor__max_depth': 30, 'regressor__max_features': 'log2', 'regressor__min_samples_leaf': 4, 'regressor__min_samples_split': 5, 'regressor__n_estimators': 50}

Best score: -0.15796425720033555

# Export

+ Save the best performing model to a pickle file.

In [ ]:
# where to save the model
pipeline_D_file = 'pipeline_D.pkl'

# Save the modelto a file
with open(pipeline_D_file, 'wb') as file:
    pickle.dump(pipeline_D_file, file)

# Close the file
file.close()

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [ ]:
import shap
data_transform = pipe_gs_D.best_estimator_.named_steps['preprocessing'].transform(X_test)

explainer = shap.explainers.Tree(
    pipe_gs_D.best_estimator_.named_steps['regressor'], 
    data_transform,
    feature_names = pipe_gs_D.best_estimator_.named_steps['preprocessing'].get_feature_names_out())


In [ ]:
shap_values = explainer(data_transform, check_additivity = False)
shap.plots.waterfall(shap_values[42], max_display = 30)
shap.plots.beeswarm(shap_values, max_display = 30)

*(Answer here.)*

The temperature, DMC (duff moisture code), coordinates, day Saturday, and FFMC (fine fuel moisture code) rank as the top features, according to these results.

The following months are the least significant, as seen in the figures: January, November, April, May, and rain.

Plotting the performance metrics (e.g., accuracy, RMSE) when utilising all features vs. selected features would allow us to see how the features affect the system.

Since the months and the rain aspects don't seem to improve the model, I would delete them.


## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.